In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *


from pathlib import Path

/home/nrhodes/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/home/nrhodes/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
PATH=Path('/home/nrhodes/data/dickens/')

In [3]:
text = (PATH/'davidcopperfield.txt').open().read()
print('corpus length:', len(text))

corpus length: 1934604


In [4]:
text[:400]

'\ufeff\nDAVID COPPERFIELD\n\n\nBy Charles Dickens\n\n\n\n               AFFECTIONATELY INSCRIBED TO\n               THE HON.  Mr. AND Mrs. RICHARD WATSON,\n               OF ROCKINGHAM, NORTHAMPTONSHIRE.\n\n\nCONTENTS\n\n\n     I.      I Am Born\n     II.     I Observe\n     III.    I Have a Change\n     IV.     I Fall into Disgrace\n     V.      I Am Sent Away\n     VI.     I Enlarge My Circle of Acquaintance\n     VII.   '

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 82


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [6]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !&(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[abcdefghijklmnopqrstuvwxy'

Map from chars to indices and back again

In [7]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

*idx* will be the data we use from now on - it simply converts all the characters to their index (based on the mapping above)

In [8]:
idx = [char_indices[c] for c in text]

idx[:10]

[81, 1, 27, 24, 45, 32, 27, 2, 26, 38]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'\ufeff\nDAVID COPPERFIELD\n\n\nBy Charles Dickens\n\n\n\n               AFFECTIONAT'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [10]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

Our inputs

In [11]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Our output

In [12]:
y = np.stack(c4_dat)

The first 4 inputs and outputs

In [13]:
x1[:4], x2[:4], x3[:4]

(array([81, 24, 27, 38]), array([ 1, 45,  2, 39]), array([27, 32, 26, 39]))

In [14]:
y[:4]

array([24, 27, 38, 28])

In [15]:
x1.shape, y.shape

((644867,), (644867,))

### Create and train model

Pick a size for our hidden state

In [16]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [17]:
n_fac = 42

In [18]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [19]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [20]:
m = Char3Model(vocab_size, n_fac).cuda()

In [21]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [22]:
opt = optim.Adam(m.parameters(), 1e-2)

In [23]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.920997   3.616276  



[array([3.61628])]

In [24]:
set_lrs(opt, 0.001)

In [25]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.699775   1.817147  



[array([1.81715])]

### Test model

In [26]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [27]:
get_next('y. ')

'I'

In [28]:
get_next('ppl')

'e'

In [31]:
get_next(' th')

'e'

In [30]:
get_next('and')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [37]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to our model.

In [48]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [49]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [50]:
xs = np.stack(c_in_dat, axis=0)

In [51]:
xs.shape

(1934596, 8)

In [52]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [53]:
xs[:cs,:cs]

array([[81,  1, 27, 24, 45, 32, 27,  2],
       [ 1, 27, 24, 45, 32, 27,  2, 26],
       [27, 24, 45, 32, 27,  2, 26, 38],
       [24, 45, 32, 27,  2, 26, 38, 39],
       [45, 32, 27,  2, 26, 38, 39, 39],
       [32, 27,  2, 26, 38, 39, 39, 28],
       [27,  2, 26, 38, 39, 39, 28, 41],
       [ 2, 26, 38, 39, 39, 28, 41, 29]])

...and this is the next character after each sequence.

In [54]:
y[:cs]

array([26, 38, 39, 39, 28, 41, 29, 32])

### Create and train model

In [55]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [56]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [57]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [58]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [59]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.941636   1.980734  



[array([1.98073])]

In [84]:
set_lrs(opt, 0.001)

In [85]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.73588  1.75103]                                 



In [92]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [93]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [94]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [95]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.81654  1.78501]                                



In [96]:
set_lrs(opt, 1e-4)

In [97]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.69008  1.69936]                                 



### Test model

In [98]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [99]:
get_next('for thos')

'e'

In [100]:
get_next('part of ')

't'

In [101]:
get_next('queens a')

'n'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [38]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [39]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [40]:
xs = np.stack(c_in_dat)
xs.shape

(241825, 8)

In [41]:
ys = np.stack(c_out_dat)
ys.shape

(241825, 8)

In [42]:
xs[:cs,:cs]

array([[81,  1, 27, 24, 45, 32, 27,  2],
       [26, 38, 39, 39, 28, 41, 29, 32],
       [28, 35, 27,  1,  1,  1, 25, 75],
       [ 2, 26, 58, 51, 68, 62, 55, 69],
       [ 2, 27, 59, 53, 61, 55, 64, 69],
       [ 1,  1,  1,  1,  2,  2,  2,  2],
       [ 2,  2,  2,  2,  2,  2,  2,  2],
       [ 2,  2,  2, 24, 29, 29, 28, 26]])

In [43]:
ys[:cs,:cs]

array([[ 1, 27, 24, 45, 32, 27,  2, 26],
       [38, 39, 39, 28, 41, 29, 32, 28],
       [35, 27,  1,  1,  1, 25, 75,  2],
       [26, 58, 51, 68, 62, 55, 69,  2],
       [27, 59, 53, 61, 55, 64, 69,  1],
       [ 1,  1,  1,  2,  2,  2,  2,  2],
       [ 2,  2,  2,  2,  2,  2,  2,  2],
       [ 2,  2, 24, 29, 29, 28, 26, 43]])

### Create and train model

In [44]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [45]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [46]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [47]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [48]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [49]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [50]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.05301    2.012837  
    1      1.864655   1.854131  
    2      1.795867   1.789595  
    3      1.753372   1.752759  



[array([1.75276])]

In [51]:
set_lrs(opt, 1e-4)

In [52]:
fit(m, md, 1, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.719005   1.734701  



[array([1.7347])]

### Identity init!

In [53]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [54]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [55]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.900178   1.876176  
    1      1.814799   1.814495  
    2      1.78488    1.775424  
    3      1.770943   1.766324  



[array([1.76632])]

In [56]:
set_lrs(opt, 1e-3)

In [57]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.679362   1.693759  
    1      1.675866   1.688033  
    2      1.667464   1.684522  
    3      1.664234   1.680948  



[array([1.68095])]

## Stateful model

### Setup

In [62]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH=Path('/home/nrhodes/data/dickens/')

TRN_PATH = 'trn'
VAL_PATH = 'val'
TRN = PATH / TRN_PATH
VAL = PATH / VAL_PATH

# Note: The student needs to practice her shell skills and prepare her own dataset before proceeding:
# - trn/trn.txt (first 80% of nietzsche.txt)
# - val/val.txt (last 20% of nietzsche.txt)

%ls {PATH}

davidcopperfield.txt  models/  trn/  val/


In [63]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(2918, 53, 1, 1494913)

### RNN

In [64]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [65]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [66]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.611406   1.639986  
    1      1.512402   1.553074  
    2      1.469829   1.526174  
    3      1.446018   1.498247  



[array([1.49825])]

In [67]:
set_lrs(opt, 1e-4)

fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.373164   1.455499  
    1      1.371756   1.450776  
    2      1.37227    1.448369  
    3      1.3738     1.446017  



[array([1.44602])]

### RNN loop

In [104]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [105]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [106]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [107]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.618142   1.646389  
    1      1.51605    1.554478  
    2      1.466418   1.52447   
    3      1.44519    1.505986  



[array([1.50599])]

### Putting it all together: LSTM

In [73]:
from fastai import sgdr

n_hidden=512

In [74]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [75]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [76]:
os.makedirs(PATH / 'models', exist_ok=True)

In [77]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.620035   1.594134  
    1      1.548822   1.522895  



[array([1.5229])]

In [78]:
on_end = lambda sched, cycle: save_model(m, PATH / f'models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.359947   1.361873  
    1      1.452329   1.433803  
    2      1.314598   1.319811  
    3      1.473417   1.453733  
    4      1.426481   1.412377  
    5      1.336148   1.338336  
    6      1.271658   1.285393  
    7      1.474852   1.455341  
    8      1.462052   1.452332  
    9      1.437847   1.426655  
    10     1.397694   1.400003  
    11     1.357319   1.35658   
    12     1.31316    1.314496  
    13     1.261445   1.280334  
    14     1.230603   1.258796  



[array([1.2588])]

In [79]:
on_end = lambda sched, cycle: save_model(m, PATH / f'models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.21652    1.25392   
    1      1.21533    1.2511    
    2      1.20758    1.248477  
    3      1.21195    1.246979  
    4      1.201187   1.24289   
    5      1.197494   1.240079  
    6      1.197302   1.238884  
    7      1.195028   1.240633  
    8      1.193652   1.237622  
    9      1.183377   1.233722  
    10     1.178493   1.23051   
    11     1.170744   1.228571  
    12     1.16136    1.22631   
    13     1.159343   1.225038  
    14     1.153263   1.224681  
    15     1.162717   1.230075  
    16     1.165779   1.227459  
    17     1.157268   1.22677   
    18     1.149207   1.224726  
    19     1.143315   1.222905  
    20     1.142363   1.22139   
    21     1.129455   1.218984  
    22     1.126839   1.218395  
    23     1.120413   1.217825  
    24     1.11784    1.216975  
    25     1.11268    1.216202  
    26     1.105899   1.215402  
    27     1.10325    1.214969  
    28     1.101869   1.214587  
    29   

[array([1.22886])]

### Test

In [68]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [69]:
get_next('for thos')

'e'

In [70]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [72]:
print(get_next_n('fourscore and seven years ago', 400))

fourscore and seven years agood less, and in the till as they she had be, as nogodany for a sable, she was busy a on the spetted that no houses of at and pleasted, obisidect, andpeggotty was quite dora was had been confidence and two along their caspetters since and sensige displadane of the holdways with a considience come and robar and all my has really, the sight of dora’s only with my arrandle on with her downs decalley-m
